<a href="https://colab.research.google.com/github/Atul-Astay/IMDB_Review_Analysis-using-SimpleRNN/blob/main/Sentiment_Analysis_SimpleRNN%26STM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow
import keras
from tensorflow.keras.layers import SimpleRNN,LSTM,GRU,Bidirectional,Dense,Embedding
from tensorflow.keras.models import Sequential

In [2]:
from tensorflow.keras.datasets import imdb

In [5]:
vocab_size = 5000
(x_train,y_train),(x_test,y_test) = imdb.load_data(num_words=vocab_size)

17464789/17464789 [==============================] - 0s 0us/step


In [6]:
print(x_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]


In [7]:
word_idx = imdb.get_word_index()

word_idx = {i:word for word,i in word_idx.items()}

1641221/1641221 [==============================] - 0s 0us/step


In [8]:
print([word_idx[i] for i in x_train[0]])

['the', 'as', 'you', 'with', 'out', 'themselves', 'powerful', 'lets', 'loves', 'their', 'becomes', 'reaching', 'had', 'journalist', 'of', 'lot', 'from', 'anyone', 'to', 'have', 'after', 'out', 'atmosphere', 'never', 'more', 'room', 'and', 'it', 'so', 'heart', 'shows', 'to', 'years', 'of', 'every', 'never', 'going', 'and', 'help', 'moments', 'or', 'of', 'every', 'chest', 'visual', 'movie', 'except', 'her', 'was', 'several', 'of', 'enough', 'more', 'with', 'is', 'now', 'current', 'film', 'as', 'you', 'of', 'mine', 'potentially', 'unfortunately', 'of', 'you', 'than', 'him', 'that', 'with', 'out', 'themselves', 'her', 'get', 'for', 'was', 'camp', 'of', 'you', 'movie', 'sometimes', 'movie', 'that', 'with', 'scary', 'but', 'and', 'to', 'story', 'wonderful', 'that', 'in', 'seeing', 'in', 'character', 'to', 'of', '70s', 'and', 'with', 'heart', 'had', 'shadows', 'they', 'of', 'here', 'that', 'with', 'her', 'serious', 'to', 'have', 'does', 'when', 'from', 'why', 'what', 'have', 'critics', 'they'

In [10]:
# max length of review
print(len(max((x_train+x_test),key = len)))
# min length of review
print(len(min((x_train+x_test),key = len)))

2697
70


In [11]:
from tensorflow.keras.preprocessing import sequence

In [12]:
max_words = 400

x_train = sequence.pad_sequences(x_train,maxlen = max_words)
x_test = sequence.pad_sequences(x_test,maxlen=max_words)

In [13]:
x_valid, y_valid = x_train[:64],y_train[:64]
x_train,y_train = x_train[64:],y_train[64:]

In [14]:
# fixing every word's embedding size to be 32
embd_len = 32
RNN_model = Sequential(name = "Simple_RNN")
RNN_model.add(Embedding(vocab_size,
                        embd_len,
                        input_length = max_words))

RNN_model.add(SimpleRNN(128,
                        activation = 'tanh',
                        return_sequences = False))
RNN_model.add(Dense(1,activation = 'sigmoid'))

In [15]:
RNN_model.summary()

Model: "Simple_RNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 400, 32)           160000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               20608     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 180737 (706.00 KB)
Trainable params: 180737 (706.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
RNN_model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

In [25]:
history = RNN_model.fit(x_train,y_train,validation_data = (x_test,y_test),batch_size = 64,epochs = 5,verbose = 'auto')

Epoch 1/5
390/390 [==============================] - 94s 241ms/step - loss: 0.5419 - accuracy: 0.7277 - val_loss: 0.6630 - val_accuracy: 0.6229
Epoch 2/5
390/390 [==============================] - 93s 238ms/step - loss: 0.5065 - accuracy: 0.7521 - val_loss: 0.5672 - val_accuracy: 0.7290
Epoch 3/5
390/390 [==============================] - 93s 239ms/step - loss: 0.4159 - accuracy: 0.8157 - val_loss: 0.5077 - val_accuracy: 0.7627
Epoch 4/5
390/390 [==============================] - 94s 242ms/step - loss: 0.4734 - accuracy: 0.7666 - val_loss: 0.6212 - val_accuracy: 0.6704
Epoch 5/5
390/390 [==============================] - 98s 252ms/step - loss: 0.4974 - accuracy: 0.7546 - val_loss: 0.6271 - val_accuracy: 0.6790


In [26]:
print(RNN_model.evaluate(x_test,y_test,verbose = 0))

[0.6271013021469116, 0.6790400147438049]
